In [1]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd



PROJECT_ROOT = os.getcwd()

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("Contenido de ./data:", os.listdir(os.path.join(PROJECT_ROOT, "data")))

PROJECT_ROOT: c:\Users\adria.flores\Documents\Projects\hacks\datathon2025-smadex
Contenido de ./data: ['sample_submission.csv', 'test', 'train']


In [2]:
import dask
import dask.dataframe as dd
dask.config.set({"dataframe.convert-string": False})

dataset_path = "./data/train"
ddf = dd.read_parquet(dataset_path)

# take ~1% of the data (so ~200k rows if you have 20M)
sample_frac = 0.0001

ddf_sample = ddf.sample(frac=sample_frac, random_state=42)

In [ ]:
# bring it to pandas for easier modeling with sklearn, etc.
df_sample = ddf_sample.compute() # consider using .head() for even smaller data
print(df_sample.shape)

(2058, 85)


In [46]:
TARGET_COLUMN = ["iap_revenue_d7"]
ROW_ID_COLUMN = ["row_id"]
OTHER_TARGET_COLUMNS = [
    "buyer_d1",
    "buyer_d7",
    "buyer_d14",
    "buyer_d28",
    "buy_d7",
    "buy_d14",
    "buy_d28",
    # other revenue horizons (excluding the main target)
    "iap_revenue_d14",
    "iap_revenue_d28",
    # registration / retention labels
    "registration",
    "retention_d1_to_d7",
    "retention_d3_to_d7",
    "retention_d7_to_d14",
    "retention_d1",
    "retention_d3",
    "retentiond7",  # note: name without underscore, as in the dataset description
]

In [47]:
y = df_sample[TARGET_COLUMN]
X = df_sample.drop(columns=TARGET_COLUMN+OTHER_TARGET_COLUMNS+ROW_ID_COLUMN)

train_row_id = df_sample[ROW_ID_COLUMN]

In [48]:
missing_ratio = X.isna().mean()
missing_ratio_sorted = missing_ratio.sort_values(ascending=False)
missing_ratio_sorted

advertiser_actions_action_count             0.985423
advertiser_actions_action_last_timestamp    0.985423
last_advertiser_action                      0.985423
last_buy_ts_category                        0.980078
last_buy                                    0.980078
                                              ...   
advertiser_bundle                           0.000000
weekday                                     0.000000
dev_os                                      0.000000
hour                                        0.000000
datetime                                    0.000000
Length: 67, dtype: float64

In [49]:
cols_to_drop = missing_ratio[missing_ratio > 0.11].index
X_rm = X.drop(columns=cols_to_drop)
print(f"Dropped {len(cols_to_drop)} columns with more than 11% missing values.")

#Chech the size of X
print("Shape of X after dropping columns:", X_rm.shape)

#Check the variables of X
print("Columns of X after dropping columns:", X_rm.columns.tolist())

Dropped 54 columns with more than 11% missing values.
Shape of X after dropping columns: (2058, 13)
Columns of X after dropping columns: ['advertiser_bundle', 'advertiser_category', 'advertiser_subcategory', 'country', 'dev_make', 'dev_model', 'dev_os', 'dev_osv', 'hour', 'release_date', 'release_msrp', 'weekday', 'datetime']


In [50]:
from sklearn.model_selection import train_test_split
import numpy as np

buyer_flag = (y > 0).astype(int)

X_train, X_val, y_train, y_val = train_test_split(
    X_rm,
    y,
    test_size=0.2,
    random_state=42,
    stratify=buyer_flag,
)

In [51]:
cat_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

for c in cat_cols:
    X_train[c] = X_train[c].astype("category")
    X_val[c] = X_val[c].astype("category")

In [52]:
y_train_log = np.log1p(y_train)
y_val_log   = np.log1p(y_val)

In [53]:
import lightgbm as lgb

model = lgb.LGBMRegressor(
    objective="regression",
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
)

model.fit(
    X_train,
    y_train_log,          # log target
    eval_set=[(X_val, y_val_log)],
    eval_metric="rmse",   # metric for logging, MSLE we'll compute ourselves
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)],
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0,000248 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 938
[LightGBM] [Info] Number of data points in the train set: 1646, number of used features: 13
[LightGBM] [Info] Start training from score 0,070682
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

,boosting_type,'gbdt'
,num_leaves,64
,max_depth,-1
,learning_rate,0.05
,n_estimators,1000
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [54]:
from sklearn.metrics import mean_squared_log_error

y_val_pred_log = model.predict(X_val, num_iteration=model.best_iteration_)
y_val_pred = np.expm1(y_val_pred_log)

# Ensure no negatives due to numeric weirdness
y_val_pred = np.clip(y_val_pred, 0, None)

msle_val = mean_squared_log_error(y_val, y_val_pred)
print("Validation MSLE:", msle_val)

Validation MSLE: 0.1690123483149397


In [ ]:
feature_cols = X_rm.columns.tolist()
cat_cols = X_rm.select_dtypes(include=["object", "category"]).columns.tolist()

In [ ]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

test_path = "./data/test"

ddf_test = dd.read_parquet(test_path)

# Keep same feature columns + row_id
ddf_test_small = ddf_test[[ROW_ID_COLUMN] + feature_cols]

df_test = ddf_test_small.compute()
print(df_test.shape)

test_row_id = df_test[ROW_ID_COLUMN]
X_test = df_test[feature_cols]